In [1]:
import pandas as pd 
import os 

In [2]:
jobs = pd.read_csv('/Users/alxander44/lda_test_10000.csv')

In [3]:
jobs.shape

(9595, 4)

In [4]:
jobs = jobs.sample(7000)

In [5]:
jobs.head()

,Unnamed: 0,0,1,2
6937,6937,17047,17049,The Team One C.O.R.E.\n\nC = Collaborative\n\n...
8185,8185,18301,18303,"A bit about us: \n\nFounded in the 1990's, we..."
1133,1133,11141,11143,SENIOR SOFTWARE ENGINEER JOB SUMMARY\n\nThis j...
2270,2270,12280,12282,Experienced programmer to work with our IT tec...
6282,6282,16381,16383,We're Nexient; we've expanded our footprint wi...


Clean Data

In [6]:
jobs = jobs.drop(columns= ['Unnamed: 0','0','1'])

In [7]:
jobs['jobs_desc'] = jobs['2']

In [8]:
jobs = jobs.drop(columns=['2'])

In [9]:
import re

jobs['jobs_desc_processed'] = jobs['jobs_desc'].map(lambda x: re.sub('[,\.!?]', '', x))

jobs['jobs_desc_processed'] = jobs['jobs_desc'].map(lambda x: x.lower())

In [10]:
jobs.head()

,jobs_desc,jobs_desc_processed
6937,The Team One C.O.R.E.\n\nC = Collaborative\n\n...,the team one c.o.r.e.\n\nc = collaborative\n\n...
8185,"A bit about us: \n\nFounded in the 1990's, we...","a bit about us: \n\nfounded in the 1990's, we..."
1133,SENIOR SOFTWARE ENGINEER JOB SUMMARY\n\nThis j...,senior software engineer job summary\n\nthis j...
2270,Experienced programmer to work with our IT tec...,experienced programmer to work with our it tec...
6282,We're Nexient; we've expanded our footprint wi...,we're nexient; we've expanded our footprint wi...


In [11]:
jobs = jobs.drop(columns=['jobs_desc'])

In [12]:
jobs.head()

,jobs_desc_processed
6937,the team one c.o.r.e.\n\nc = collaborative\n\n...
8185,"a bit about us: \n\nfounded in the 1990's, we..."
1133,senior software engineer job summary\n\nthis j...
2270,experienced programmer to work with our it tec...
6282,we're nexient; we've expanded our footprint wi...


In [19]:
import numpy as np 
import gensim

In [20]:
# Tokenize the sentences into a list of words

from gensim.utils import simple_preprocess

In [21]:
def sentence_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True)) 

# deacc all that does is further remove punctuations doing this is easier than
# using a more complex re function

data = jobs['jobs_desc_processed'].values.tolist()
data_words = list(sentence_to_words(data))

In [22]:
print(data_words[0][:200])

['the', 'team', 'one', 'collaborative', 'builds', 'and', 'sustains', 'strong', 'genuine', 'relationships', 'with', 'people', 'of', 'all', 'levels', 'and', 'takes', 'positive', 'steps', 'to', 'extend', 'network', 'and', 'cultivate', 'connections', 'optimism', 'is', 'utterly', 'reliable', 'always', 'delivering', 'on', 'promises', 'and', 'maintains', 'the', 'highest', 'standards', 'of', 'honesty', 'and', 'integrity', 'results', 'driven', 'acts', 'strategically', 'to', 'exceed', 'goals', 'and', 'has', 'the', 'ability', 'to', 'adapt', 'to', 'changes', 'in', 'process', 'and', 'execution', 'entrepreneurial', 'constantly', 'looking', 'for', 'and', 'identifying', 'actionable', 'and', 'thought', 'provoking', 'opportunities', 'for', 'the', 'company', 'and', 'executing', 'them', 'general', 'summary', 'the', 'creative', 'technologist', 'understands', 'the', 'business', 'of', 'advertising', 'marketing', 'and', 'branding', 'and', 'takes', 'creative', 'strategic', 'and', 'people', 'centric', 'view', '

Bi-grams and Tri-grams

In [23]:
# Bi-grams and Tri-grams  are words that frequently occur together in a the document
# for ex data science or machine learning  or machine learning engineer 
# https://radimrehurek.com/gensim/models/phrases.html

In [24]:
# build a bigram and trigram model
# The higher the values of these param, the harder it is for words to be combined.
# the higher the threshold the fewer the phrases that it models for 

bigram = gensim.models.Phrases(data_words,min_count=5, threshold=100)
trigram = gensim.models.Phrases(data_words,min_count=5, threshold=100)

Now that we have the models built lets run them

In [25]:
from gensim.models.phrases import Phrases, Phraser
# Fast way to get the sentences clubbed as bi and trigrams

bigram_model = Phraser(bigram)
trigram_model = Phraser(trigram)

Remove STOPWORDS

In [26]:
# you can import spacy or NLTK 

In [27]:
%%time
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject','\n\n','\xa0','\n\n\n','this…your','-', '|', '\n','&','ul','li','position','jobs','job','etc','work','business','employment',
                   'employer','employee','applicant','good','use','must','work'])

# Creating a function for the stop words bigrams trigrams and lemmatization 

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))if word
             not in stop_words] for doc in texts]

def make_bigrams(texts):
    return[bigram_model[doc] for doc in texts]

def make_trigram(texts):
    return[trigram_model[bigram_model[doc]]for doc in texts]
           
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

CPU times: user 207 ms, sys: 73.9 ms, total: 281 ms
Wall time: 460 ms


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alxander44/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!




Time to call the functions in the same order


In [28]:
%%time
import spacy 
nlp = spacy.load("en_core_web_lg")

# Lets remove the stop words with the stopword function that we created above
data_words_nostops = remove_stopwords(data_words)

# Now lets form the Bigrams
data_words_bigram = make_bigrams(data_words_nostops)

data_words_trigram = make_trigram(data_words_bigram)

# Time lemmatization we are only going to keep the NOUNS ADJ VB AND ADV (May come back to this and change it though)
data_lemmatized = lemmatization(data_words_trigram, allowed_postags= ['NOUN', 'ADJ', 'VERB', 'ADV'])

CPU times: user 4min 25s, sys: 19.6 s, total: 4min 45s
Wall time: 4min 47s


In [27]:
# now let take a little peak at what we have done so far 
print(data_words_trigram[0][:100])

['title', 'react', 'node_js', 'developer', 'location', 'jacksonville_fl', 'duration_months', 'contract', 'required', 'experience', 'years', 'node', 'development', 'heavy', 'back', 'priority', 'candidate', 'able', 'build', 'back', 'end', 'databases', 'utilize', 'redux', 'github', 'rest', 'services', 'developer', 'api', 'years', 'development', 'programming', 'coding', 'professional', 'experience', 'required', 'education', 'mid', 'level', 'developer', 'years', 'minimum', 'years', 'primarily', 'node', 'back', 'end', 'react', 'development', 'college', 'level', 'projects', 'count', 'experience', 'need', 'real', 'experience', 'bachelors', 'masters', 'degree', 'engineering', 'technology', 'priority', 'skills', 'redux', 'github', 'rest', 'services', 'api', 'node', 'past', 'candidates', 'long', 'enough', 'java', 'node', 'react', 'years', 'experience', 'node', 'react', 'needed', 'person', 'needs', 'able', 'build', 'back', 'end', 'databases', 'manager', 'looking', 'nodejs', 'experience', 'reactjs'

In [28]:
print(data_words[0][:100])

['job', 'title', 'react', 'node', 'js', 'developer', 'location', 'jacksonville', 'fl', 'duration', 'months', 'contract', 'required', 'experience', 'must', 'have', 'years', 'of', 'node', 'development', 'heavy', 'back', 'is', 'priority', 'candidate', 'must', 'be', 'able', 'to', 'build', 'back', 'end', 'databases', 'utilize', 'redux', 'github', 'rest', 'services', 'developer', 'api', 'years', 'of', 'it', 'development', 'programming', 'coding', 'professional', 'work', 'experience', 'required', 'education', 'mid', 'level', 'developer', 'years', 'minimum', 'of', 'years', 'of', 'primarily', 'node', 'back', 'end', 'and', 'react', 'development', 'college', 'level', 'projects', 'do', 'not', 'count', 'as', 'experience', 'need', 'real', 'work', 'experience', 'bachelors', 'or', 'masters', 'degree', 'in', 'engineering', 'technology', 'etc', 'priority', 'skills', 'redux', 'github', 'rest', 'services', 'api', 'node', 'past', 'candidates', 'did', 'not', 'have', 'long']


In [29]:
print(data_words_trigram[0][:100])

['title', 'react', 'node_js', 'developer', 'location', 'jacksonville_fl', 'duration_months', 'contract', 'required', 'experience', 'years', 'node', 'development', 'heavy', 'back', 'priority', 'candidate', 'able', 'build', 'back', 'end', 'databases', 'utilize', 'redux', 'github', 'rest', 'services', 'developer', 'api', 'years', 'development', 'programming', 'coding', 'professional', 'experience', 'required', 'education', 'mid', 'level', 'developer', 'years', 'minimum', 'years', 'primarily', 'node', 'back', 'end', 'react', 'development', 'college', 'level', 'projects', 'count', 'experience', 'need', 'real', 'experience', 'bachelors', 'masters', 'degree', 'engineering', 'technology', 'priority', 'skills', 'redux', 'github', 'rest', 'services', 'api', 'node', 'past', 'candidates', 'long', 'enough', 'java', 'node', 'react', 'years', 'experience', 'node', 'react', 'needed', 'person', 'needs', 'able', 'build', 'back', 'end', 'databases', 'manager', 'looking', 'nodejs', 'experience', 'reactjs'

In [30]:
print(data_lemmatized[0][:100])

['title', 'react', 'node_js', 'developer', 'location', 'jacksonville_fl', 'duration_month', 'contract', 'require', 'experience', 'year', 'node', 'development', 'heavy', 'back', 'priority', 'candidate', 'able', 'build', 'end', 'database', 'utilize', 'rest', 'service', 'developer', 'api', 'year', 'development', 'programming', 'code', 'professional', 'experience', 'require', 'education', 'mid', 'level', 'developer', 'year', 'minimum', 'year', 'primarily', 'node', 'back', 'end', 'react', 'development', 'college', 'level', 'project', 'count', 'experience', 'need', 'real', 'experience', 'bachelor', 'master', 'degree', 'engineering', 'technology', 'priority', 'skill', 'github', 'rest', 'service', 'api', 'node', 'past', 'candidate', 'long', 'enough', 'java', 'node', 'react', 'year', 'experience', 'node', 'react', 'need', 'person', 'need', 'able', 'build', 'end', 'database', 'manager', 'look', 'nodej', 'experience', 'reactj']


LDA Topic Model Dictionary id2word


In [29]:
import gensim.corpora as corpora

# Creating the dictionary 
id2word = corpora.Dictionary(data_lemmatized)

# Doing a little filter to get better results from the model 

id2word.filter_extremes(no_below=5, no_above=.90)

# Creating the corpus (REMEMBER THAT THE Corpus is a collection of documents)

corpus = [id2word.doc2bow(text) for text in data_lemmatized]

In [30]:
# LETS take a look 

print(corpus[0])

[(0, 5), (1, 2), (2, 1), (3, 2), (4, 3), (5, 1), (6, 2), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 16), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 3), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 4), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 2), (75, 2), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 7), (99, 2), (100, 1), (101, 3), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 4), (109, 1), (110, 1)

In [31]:
# count inquiry 

print(len(id2word))
len(id2word.keys()) # the amount of words that we have in our data 

9821


9821

In [46]:
id2word.save('id2word')

/Users/alxander44/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL




TIME TO BUILD THE LDA MODEL !!!!!


In [32]:
# lda MODEL

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=17, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='asymmetric',
                                           eta=.9,
                                           per_word_topics=True)

# chunksize is how many docs get passed thru the model at a time in the training algorithm
# passes are like epochs in NNs

In [33]:
# view topics of the lda model 
lda_model.print_topics()

[(0,
  '0.029*"status" + 0.028*"disability" + 0.022*"sexual_orientation" + 0.020*"age" + 0.020*"equal_opportunity" + 0.019*"gender_identity" + 0.015*"national_origin" + 0.011*"protected_veteran" + 0.010*"applicant" + 0.010*"without_regard"'),
 (1,
  '0.081*"datum" + 0.019*"data" + 0.016*"analysis" + 0.016*"report" + 0.014*"analytic" + 0.013*"analyst" + 0.013*"sql" + 0.012*"skill" + 0.010*"database" + 0.009*"tool"'),
 (2,
  '0.053*"salesforce" + 0.014*"sap" + 0.011*"unisys" + 0.009*"ibm" + 0.008*"lightning" + 0.007*"critical" + 0.007*"force" + 0.007*"mission" + 0.006*"cloud" + 0.005*"apex"'),
 (3,
  '0.020*"system" + 0.019*"engineering" + 0.012*"test" + 0.012*"engineer" + 0.010*"design" + 0.010*"product" + 0.010*"software" + 0.009*"require" + 0.007*"quality" + 0.007*"requirement"'),
 (4,
  '0.025*"development" + 0.022*"application" + 0.021*"web" + 0.018*"developer" + 0.016*"year" + 0.015*"design" + 0.013*"skill" + 0.013*"java" + 0.012*"javascript" + 0.011*"use"'),
 (5,
  '0.019*"company

In [34]:
words =[re.findall(r'"([^"]*)"',t[1]) for t in lda_model.print_topics()]

In [35]:
topics = [' '.join(t[0:5]) for t in words]

In [36]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
status disability sexual_orientation age equal_opportunity

------ Topic 1 ------
datum data analysis report analytic

------ Topic 2 ------
salesforce sap unisys ibm lightning

------ Topic 3 ------
system engineering test engineer design

------ Topic 4 ------
development application web developer year

------ Topic 5 ------
company team benefit opportunity look

------ Topic 6 ------
system technical team project development

------ Topic 7 ------
deloitte guest learn people offer

------ Topic 8 ------
security network server infrastructure support

------ Topic 9 ------
team customer product sale client

------ Topic 10 ------
require ability project include service

------ Topic 11 ------
software team technology engineer cloud

------ Topic 12 ------
design user product ux team

------ Topic 13 ------
team member perform hand kiewit

------ Topic 14 ------
patient care health clinical healthcare

------ Topic 15 ------
ibkr interactive_broker broker trade w

In [37]:
# compute coherence Score

from gensim.models import CoherenceModel


coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.49637179690802924


In [ ]:
# supporting function
# def compute_coherence_values(corpus, dictionary, k, a, b):
    
   # lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                         #  id2word=id2word,
                                         #  num_topics=20, 
                                         #  random_state=100,
                                         #  chunksize=100,
                                         #  passes=10,
                                         #  alpha=a,
                                         #  eta=b,
                                         #  per_word_topics=True)
    
    # coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    # return coherence_model_lda.get_coherence()

In [ ]:
## import numpy as np
#import tqdm
#grid = {}
#grid['Validation_Set'] = {}
# Topics range
#min_topics = 8
#max_topics = 21
#step_size = 1
#topics_range = range(min_topics, max_topics, step_size)


In [ ]:
# Alpha parameter
#alpha = list(np.arange(0.01, 1, 0.3))
#alpha.append('symmetric')
#alpha.append('asymmetric')
# Beta parameter
#beta = list(np.arange(0.01, 1, 0.3))
#beta.append('symmetric')

In [ ]:
# Validation sets
#num_of_docs = len(corpus)
#corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
              # gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.15)), 
              # corpus]
#corpus_title = ['15% Corpus', '100% Corpus']
#model_results = {'Validation_Set': [],
 #                'Topics': [],
 #                'Alpha': [],
 #                'Beta': [],
 #                'Coherence': []
 #               }

In [ ]:

# Can take a long time to run
#if 1 == 1:
 #   pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
  #  for i in range(len(corpus_sets)):
        # iterate through number of topics
   #     for k in topics_range:
            # iterate through alpha values
    #        for a in alpha:
                # iterare through beta values
     #           for b in beta:
                    # get the coherence score for the given parameters
                 #   cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                  #                                k=k, a=a, b=b)
                    # Save the model results
                  #  model_results['Validation_Set'].append(corpus_title[i])
                  #  model_results['Topics'].append(k)
                  #  model_results['Alpha'].append(a)
                  #  model_results['Beta'].append(b)
                  #  model_results['Coherence'].append(cv)
                    
                  #  pbar.update(1)
    #pd.DataFrame(model_results).to_csv('lda_tuning_results2.csv', index=False)
    #pbar.close()


In [49]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
fmri = sns.load_dataset("fmri")

In [ ]:
# lda_plot = pd.read_csv('lda_tuning_results2.csv')

In [ ]:
# lda_plot.sample(3)

In [41]:
# test = lda_plot.sample(3)

In [42]:
# test = test.drop(columns=['Validation_Set'])

In [ ]:
# test

In [44]:
# lda_plot.sort_values(by=['Coherence','Topics'],ascending=False).head(20)


,Validation_Set,Topics,Alpha,Beta,Coherence
418,100% Corpus,6,asymmetric,0.9099999999999999,0.419363
448,100% Corpus,7,asymmetric,0.9099999999999999,0.416761
478,100% Corpus,8,asymmetric,0.9099999999999999,0.416261
358,100% Corpus,4,asymmetric,0.9099999999999999,0.413645
538,100% Corpus,10,asymmetric,0.9099999999999999,0.410064
388,100% Corpus,5,asymmetric,0.9099999999999999,0.409432
383,100% Corpus,5,symmetric,0.9099999999999999,0.408563
328,100% Corpus,3,asymmetric,0.9099999999999999,0.407185
394,100% Corpus,6,0.01,symmetric,0.405962
468,100% Corpus,8,0.9099999999999999,0.9099999999999999,0.403997


In [45]:
#sns.lineplot(x='Coherence', y='Alpha', data=test)
#ax.show()



Final Model



In [ ]:
#lda_plot.sort_values(by=['Coherence','Topics'],ascending=False).head(20)


In [45]:
import pyLDAvis.gensim
import pickle
import pyLDAvis

pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model,corpus,id2word)

LDAvis_prepared

/Users/alxander44/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.241020  0.014850       1        1  20.089602
4      0.179051 -0.129596       2        1  12.336979
9      0.151343  0.038934       3        1  12.073631
11     0.150209 -0.093402       4        1  10.564448
10     0.091455  0.130310       5        1   9.555889
5      0.068525  0.030795       6        1   9.474989
12     0.124079 -0.049512       7        1   6.000378
1      0.116494 -0.018908       8        1   5.864853
3      0.057986  0.022113       9        1   5.132487
0     -0.080791  0.179639      10        1   3.970880
14    -0.043060  0.134664      11        1   2.055353
8     -0.108806 -0.113681      12        1   1.352542
7     -0.178619 -0.002877      13        1   0.728189
2     -0.199497 -0.041526      14        1   0.422967
13    -0.159906 -0.020073      15        1   0.295374
15    -0.204669 -0.041356      16        1   0.057337
16    -0.204815 -0.040374      17        1   0.024106, topic_info=     Category          Freq                   Term         Total  loglift  \
53    Default  10917.000000                  datum  10917.000000  30.0000   
58    Default  14887.000000                 design  14887.000000  29.0000   
486   Default   6032.000000                   user   6032.000000  28.0000   
633   Default   2626.000000               security   2626.000000  27.0000   
348   Default   6129.000000                    web   6129.000000  26.0000   
336   Default  10399.000000               software  10399.000000  25.0000   
227   Default  10758.000000                 system  10758.000000  24.0000   
182   Default   9417.000000                product   9417.000000  23.0000   
638   Default   2567.000000                 status   2567.000000  22.0000   
652   Default   3443.000000                  cloud   3443.000000  21.0000   
731   Default   2741.000000             disability   2741.000000  20.0000   
525   Default   4503.000000            engineering   4503.000000  19.0000   
397   Default   5128.000000               engineer   5128.000000  18.0000   
363   Default  10681.000000            application  10681.000000  17.0000   
342   Default   7123.000000                   test   7123.000000  16.0000   
230   Default  17154.000000                   team  17154.000000  15.0000   
224   Default   8915.000000                support   8915.000000  14.0000   
63    Default   5323.000000              developer   5323.000000  13.0000   
64    Default  14352.000000            development  14352.000000  12.0000   
360   Default   3950.000000               analysis   3950.000000  11.0000   
1178  Default   3260.000000                   java   3260.000000  10.0000   
198   Default   3772.000000                 report   3772.000000   9.0000   
150   Default   1428.000000                network   1428.000000   8.0000   
589   Default   2521.000000                   data   2521.000000   7.0000   
335   Default   9588.000000                service   9588.000000   6.0000   
338   Default   3028.000000                    sql   3028.000000   5.0000   
1980  Default   2728.000000                   sale   2728.000000   4.0000   
595   Default   2296.000000      equal_opportunity   2296.000000   3.0000   
635   Default   1610.000000     sexual_orientation   1610.000000   2.0000   
334   Default   1811.000000                 server   1811.000000   1.0000   
...       ...           ...                    ...           ...      ...   
3169  Topic17      0.111336             incredibly     29.330074   2.7567   
8833  Topic17      0.053629                tightly     14.560029   2.7265   
7479  Topic17      0.040497              fluctuate     12.139421   2.6275   
2809  Topic17      0.040497        tomcat_weblogic     12.195585   2.6229   
2583  Topic17      0.040497          flex_spending     12.213184   2.6214   
6863  Topic17      0.040497  greenwich_connecticut     12.252868   2.6182   
6867 

In [47]:
# pyLDAvis.enable_notebook()

# LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_asymmetric_8,corpus,id2word)

# LDAvis_prepared


In [48]:

# pyLDAvis.enable_notebook()

# LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_asymmetric_10,corpus,id2word)

# LDAvis_prepared


In [38]:
from gensim.utils import SaveLoad

In [39]:
import pickle

In [40]:
from gensim import corpora, models, similarities

In [41]:
import pickle


In [42]:
with open('lda_model', 'wb') as f:
    pickle.dump(lda_model,f)

In [43]:
with open('lda_model', 'rb') as f:
    lda_m = pickle.load(f)

In [44]:
lda_model.save('lda_model_final')

/Users/alxander44/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [86]:
lda_model_asymmetric_8.save('lda_model_topic_8')

/Users/alxander44/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [87]:
import os
import tempfile

In [90]:
with tempfile.NamedTemporaryFile() as tmp:
    lda_model_asymmetric_8.save('this_IS_a_test')

In [91]:
loaded_lda_model = models.LdaModel.load('this_IS_a_test')

In [92]:
loaded_lda_model

In [ ]:
vector = loaded_lda_model[data_word]